# Alpha GO Zeros (part2: 训练agent)


In [1]:
# 环境配置
%cd /playground/sgd_deep_learning/sgd_rl/go
import sys
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rl


In [2]:
import os
import torch

from dlgo import zero

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 数据 模型存放目录
data_home_path = 'data/agz/'
if not os.path.exists(data_home_path):
    os.makedirs(data_home_path)

class args:
    learning_agent= data_home_path + 'agent_checkpoint.pth'
    experience = [data_home_path + 'experience.pth'] # 训练数据可以保存在多个文件中，递归执行训练
    agent_out = data_home_path + 'agent_checkpoint_update.pth'
    
    board_size = 9 # 缩小计算量, 保证算法的验证速度
    num_games = 10 # 每轮迭代只收集10games的数据
    lr=0.001
    batch_size =512 # batch_size
    
# 全局变量
global BOARD_SIZE
BOARD_SIZE = args.board_size

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("model file exists: ", os.path.exists(args.learning_agent))
print("device:", device)

model file exists:  True
device: cuda


In [4]:
def main():
    learning_agent_filename = args.learning_agent
    updated_agent_filename = args.agent_out
    
    experience_files = args.experience
    
    learning_rate = args.lr
    batch_size = args.batch_size
    
    # init agent object
    encoder = zero.ZeroEncoder(BOARD_SIZE)  # 注意这里的转变
    model = zero.agznet(input_channel_num=encoder.num_planes(), 
                            moves_num=encoder.num_moves())

    # 训练阶段只需要一个agent train self._model 即可
    learning_agent = None
    if not os.path.exists(learning_agent_filename): # check_point不存在
        learning_agent = zero.ZeroAgent(model, encoder, rounds_per_move=20, c=2.0, device=device) 
    else: 
        learning_agent = zero.load_agent(model, encoder, learning_agent_filename, rounds_per_move=10, c=2.0, device=device)
    assert learning_agent is not None

    # 读取数据训练
    for exp_filename in experience_files:
        exp_buffer = zero.load_experience(exp_filename)
        
        learning_agent.train(
            exp_buffer,
            lr=learning_rate,
            batch_size=batch_size)

    # 模型再保存
    learning_agent.serialize(updated_agent_filename)

    # todo 
    # 会产生一堆的checkpoint和 训练数据，这些文件保存的名字怎么选择
    # 怎么自动的去迭代这个训练过程，非要人工介入的时候再暂停人工介入
main()

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


iter: 1 batch_size 512
iter: 2 batch_size 512
iter: 3 batch_size 40
